In [2]:
from abc import ABC, abstractmethod

class PaymentProvider(ABC):
    @abstractmethod
    def make_payment(self, amount: float, method: str): pass


class PaymentMethod(ABC):
    def __init__(self, provider: PaymentProvider): ...
    @abstractmethod
    def pay(self, amount: float): pass


In [5]:
from abc import ABC, abstractmethod
import random

class PaymentMethod(ABC):        
    @abstractmethod
    def pay(self, amount: float): pass

class PaymentProvider(ABC):
    def __init__(self, paymentMethod: PaymentMethod):
        self.paymentMethod = paymentMethod
        
    @abstractmethod
    def make_payment(self, amount: float): pass


class Stripe(PaymentProvider):
    def make_payment(self, amount):
        return "[Stripe] Charged "+self.paymentMethod.pay(amount)

class RazorPay(PaymentProvider):
    def make_payment(self, amount):
        return "[Razorpay] Payment "+self.paymentMethod.pay(amount)
        
class PayPal(PaymentProvider):
    def make_payment(self, amount):
        return "[PayPal] Paid "+self.paymentMethod.pay(amount)
        

class CreditCard(PaymentMethod):
    def pay(self,amount):
        IS_DONE = random.randint(0,1)
        if IS_DONE:
            return f"Failed {amount}$ due to opt issue!"
        return f"{amount}$ sucess via credit card"
      
class UPI(PaymentMethod):
    def pay(self,amount):
        IS_DONE = random.randint(0,1)
        if IS_DONE:
            return f"Failed {amount}$ due to password wrong!"
        return f"{amount}$ sucess via UPI"

class Wallet(PaymentMethod):
    def pay(self,amount):
        IS_DONE = random.randint(0,1)
        if IS_DONE:
            return f"Failed {amount}$ due to pin incorrect!"
        return f"{amount}$ sucess via wallet"
    
PayPal(Wallet()).make_payment(5)
        

'[PayPal] Paid Failed 5$ due to pin incorrect!'

In [ ]:
# -- Drop and recreate the database
# DROP DATABASE IF EXISTS startup1;
# CREATE DATABASE startup1 CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
# USE startup1;

# -- 1. Master data

# DROP TABLE IF EXISTS cities;
# CREATE TABLE cities (
#   id          INT              NOT NULL AUTO_INCREMENT,
#   name        VARCHAR(255)     NOT NULL,
#   region      VARCHAR(255),
#   country     VARCHAR(255)     NOT NULL,
#   PRIMARY KEY (id),
#   UNIQUE KEY ux_cities_name_region_country (name, region, country)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# DROP TABLE IF EXISTS currencies;
# CREATE TABLE currencies (
#   code        CHAR(3)          NOT NULL,
#   name        VARCHAR(100)     NOT NULL,
#   symbol      VARCHAR(10),
#   PRIMARY KEY (code)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# -- 2. Users & sessions

# DROP TABLE IF EXISTS users;
# CREATE TABLE users (
#   id          INT              NOT NULL AUTO_INCREMENT,
#   username    VARCHAR(100)     NOT NULL,
#   email       VARCHAR(255)     NOT NULL,
#   first_name  VARCHAR(100),
#   last_name   VARCHAR(100),
#   mobile      VARCHAR(20),
#   city_id     INT,
#   gender      ENUM('M','F','Other'),
#   birthday    DATE,
#   created_at  TIMESTAMP        NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   UNIQUE KEY ux_users_username (username),
#   UNIQUE KEY ux_users_email    (email),
#   INDEX idx_users_city         (city_id),
#   CONSTRAINT fk_users_city     FOREIGN KEY (city_id) REFERENCES cities(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# DROP TABLE IF EXISTS sessions;
# CREATE TABLE sessions (
#   id          BINARY(16)       NOT NULL PRIMARY KEY
#     DEFAULT (UUID_TO_BIN(UUID())),
#   user_id     INT              NOT NULL,
#   started_at  TIMESTAMP        NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   ended_at    TIMESTAMP NULL,
#   INDEX idx_sessions_user     (user_id),
#   CONSTRAINT fk_sessions_user FOREIGN KEY (user_id) REFERENCES users(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# -- 3. User inputs

# DROP TABLE IF EXISTS user_inputs;
# CREATE TABLE user_inputs (
#   id             INT           NOT NULL AUTO_INCREMENT,
#   session_id     BINARY(16)    NOT NULL,
#   interest_box   TEXT          NOT NULL,
#   city_id        INT           NOT NULL,
#   budget         DECIMAL(10,2),
#   currency_code  CHAR(3)       NOT NULL,
#   created_at     TIMESTAMP     NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_ui_city_budget  (city_id, budget),
#   CONSTRAINT fk_ui_session  FOREIGN KEY (session_id)    REFERENCES sessions(id),
#   CONSTRAINT fk_ui_city     FOREIGN KEY (city_id)       REFERENCES cities(id),
#   CONSTRAINT fk_ui_currency FOREIGN KEY (currency_code) REFERENCES currencies(code)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# -- 4. Places & reviews

# DROP TABLE IF EXISTS places;
# CREATE TABLE places (
#   id             INT           NOT NULL AUTO_INCREMENT,
#   name           VARCHAR(255)  NOT NULL,
#   address        TEXT,
#   city_id        INT           NOT NULL,
#   geog           POINT         NULL,
#   avg_cost_pp    DECIMAL(10,2),
#   description    TEXT,
#   created_at     TIMESTAMP     NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_places_city    (city_id),
#   SPATIAL INDEX spx_places_geog (geog),
#   CONSTRAINT fk_places_city FOREIGN KEY (city_id) REFERENCES cities(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# DROP TABLE IF EXISTS place_reviews;
# CREATE TABLE place_reviews (
#   id             INT           NOT NULL AUTO_INCREMENT,
#   place_id       INT           NOT NULL,
#   user_id        INT           NOT NULL,
#   rating         FLOAT         NOT NULL,
#   review_text    TEXT,
#   created_at     TIMESTAMP     NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_pr_place       (place_id),
#   INDEX idx_pr_user_place  (user_id, place_id),
#   CONSTRAINT fk_pr_place   FOREIGN KEY (place_id) REFERENCES places(id),
#   CONSTRAINT fk_pr_user    FOREIGN KEY (user_id)  REFERENCES users(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# -- 5. AI responses & feedback

# DROP TABLE IF EXISTS ai_responses;
# CREATE TABLE ai_responses (
#   id              INT           NOT NULL AUTO_INCREMENT,
#   session_id      BINARY(16)    NOT NULL,
#   user_input_id   INT           NOT NULL,
#   place_id        INT           NOT NULL,
#   why_recommended TEXT          NOT NULL,
#   cost            DECIMAL(10,2),
#   place_timing    TIMESTAMP,
#   created_at      TIMESTAMP     NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_ar_session    (session_id),
#   INDEX idx_ar_user_input (user_input_id),
#   CONSTRAINT fk_ar_session     FOREIGN KEY (session_id)     REFERENCES sessions(id),
#   CONSTRAINT fk_ar_user_input  FOREIGN KEY (user_input_id)  REFERENCES user_inputs(id),
#   CONSTRAINT fk_ar_place       FOREIGN KEY (place_id)        REFERENCES places(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# DROP TABLE IF EXISTS ai_response_feedback;
# CREATE TABLE ai_response_feedback (
#   id               INT          NOT NULL AUTO_INCREMENT,
#   ai_response_id   INT          NOT NULL,
#   rating           FLOAT        NOT NULL,
#   comments         TEXT,
#   created_at       TIMESTAMP    NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_arf_arid (ai_response_id),
#   CONSTRAINT fk_arf_arid FOREIGN KEY (ai_response_id) REFERENCES ai_responses(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

# -- 6. App-level feedback

# DROP TABLE IF EXISTS app_feedback;
# CREATE TABLE app_feedback (
#   id            INT           NOT NULL AUTO_INCREMENT,
#   user_id       INT           NOT NULL,
#   rating        FLOAT         NOT NULL,
#   feedback      TEXT,
#   suggestions   TEXT,
#   created_at    TIMESTAMP     NOT NULL DEFAULT CURRENT_TIMESTAMP,
#   PRIMARY KEY (id),
#   INDEX idx_af_user (user_id),
#   CONSTRAINT fk_af_user FOREIGN KEY (user_id) REFERENCES users(id)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;